In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time
import os

In [ ]:
def get_urls_from_main(url):
    SCROLL_PAUSE_TIME = 0.5

    driver = webdriver.Chrome()
    driver.set_page_load_timeout(5)

    driver.get(url)

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, "html.parser")

    div_class = "max-w-full lg:h-64 h-40 rounded-md relative overflow-hidden uk-transition-toggle"

    hrefs = []

    for link in soup.find_all("div", class_=div_class):
        hrefs.append(link.parent.get("href"))

    driver.close()
    return hrefs

In [ ]:
def get_urls_to_download(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    source = None
    
    if soup.find("video"):
        source = soup.find("video").find("source").get("src")
    else:
        source = soup.find("a", class_="uk-align-center").find("img").get("src")

    return source

In [ ]:
def download(url):
    name = url.split("/")[-1]
    ext = url[-4:]

    if ext == ".mp4":
        resp = requests.get(url, stream=True)
        with open(f"{name}", "wb") as f:
            f.write(resp.content)

    else:
        resp = requests.get(url)
        with open(f"{name}", "wb") as f:
            print(f"writing file {name}")
            f.write(resp.content)

    print("done")

In [ ]:
def main(url=None):
    if url is None:
        print("input url")
        return
    
    folder_name = url.split('/')[-1] if url.split('/')[-1] != "" else url.split("/")[-2]
    
    
    try:
        if not os.path.exists(f'./{folder_name}'):
            os.mkdir(f'./{folder_name}')
    except:
        folder_name = "img"
        if not os.path.exists(f'./{folder_name}'):
            os.mkdir(f'./{folder_name}')
            
    os.chdir(f'./{folder_name}')
    
    try: 
        hrefs= get_urls_from_main(url)
    except:
        print("input correct url")
    else:
        missing = len(hrefs)
        for url in hrefs:
            print(f"missing ==> {missing} files")
            source = get_urls_to_download(url)
            if source == None:
                print("failed to get source", url)
            else:
                try:
                    download(source)
                except:
                    print("Download failed", source)
            missing -= 1
            
    os.chdir(f'../')

In [ ]:
from dotenv import load_dotenv

load_dotenv()

URL = os.getenv('URL')

main(URL)
